In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
#from solver import *
import os

import torch
from torch import optim
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F
import sys

In [2]:
os.chdir('C:/Users/micka/Documents/PythonProgram/Images')
img = cv2.imread('sudoku.jpeg')
cv2.imshow("Contour", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# Load an color image in grayscale
img = cv2.imread('sudoku.jpeg')

In [4]:
def find_board(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to gray
    bfilter=cv2.bilateralFilter(gray,13,20,20) #remove noise while keeping edges sharp
    edged=cv2.Canny(bfilter,100,200) #find edges
    cv2.imshow("Contour", edged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    newimg = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 3)
    cv2.imshow("Contour", newimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    contours=sorted(contours, key=cv2.contourArea, reverse = True)[:15]
    location=None
    # Finds rectangular contour
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 15, True)
        if len(approx) == 4:
            location = approx
            break
    result = get_perspective(img, location)
    return result, location


In [5]:
def find_board(img):
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 150, 255, 0)
    keypoints= cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    newimg = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 3)
    cv2.imshow("Contour", newimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    contours=sorted(contours, key=cv2.contourArea, reverse = True)[:15]
    location=None
    # Finds rectangular contour
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 15, True)
        if len(approx) == 4:
            location = approx
            break
    result = get_perspective(img, location)
    return result, location

In [92]:
find_board(img)

(array([[[175, 178, 199],
         [154, 157, 178],
         [135, 138, 159],
         ...,
         [158, 161, 182],
         [161, 164, 185],
         [164, 167, 188]],
 
        [[173, 176, 197],
         [152, 155, 176],
         [132, 135, 156],
         ...,
         [149, 152, 173],
         [153, 156, 177],
         [155, 158, 179]],
 
        [[170, 173, 194],
         [150, 153, 174],
         [129, 132, 153],
         ...,
         [140, 143, 164],
         [144, 147, 168],
         [146, 149, 170]],
 
        ...,
 
        [[118, 114, 125],
         [102,  98, 109],
         [ 85,  81,  92],
         ...,
         [ 77,  69,  92],
         [ 93,  85, 108],
         [122, 114, 137]],
 
        [[126, 122, 133],
         [113, 109, 120],
         [100,  96, 107],
         ...,
         [ 80,  72,  95],
         [ 97,  89, 112],
         [124, 116, 139]],
 
        [[144, 140, 151],
         [136, 132, 143],
         [127, 123, 134],
         ...,
         [ 92,  84, 107],
  

In [6]:
    contours=sorted(contours, key=cv2.contourArea, reverse = True)[:15]
    location=None
    # Finds rectangular contour
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 15, True)
    if len(approx) == 4:
        location = approx
        break
    result = get_perspective(img, location)
    return result, location

NameError: name 'contours' is not defined

In [7]:
def get_perspective(img, location, height = 900, width = 900):
    print(location)
    """Takes an image and location of an interesting region.
    And return the only selected region with a perspective transformation"""
    pts1 = np.float32([location[3], location[0], location[2], location[1]])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    # Apply Perspective Transform Algorithm
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    result = cv2.warpPerspective(img, matrix, (width, height))
    return result

In [8]:
board, location = find_board(img)
cv2.imshow("Board", board)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[405  90]]

 [[438 392]]

 [[ 52 404]]

 [[ 80 102]]]


In [9]:
# split the board into 81 individual images
def split_boxes(board):
    """Takes a sudoku board and split it into 81 cells.
    each cell contains an element of that board either given or an empty cell."""
    rows = np.vsplit(board,9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r,9)
        for box in cols:
            box = cv2.resize(box, (100, 100))/255.0
            #cv2.imshow("Splitted block", box)
            #cv2.waitKey(50)
            boxes.append(box)
    #cv2.imshow("Box", boxes[1])
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()    
    return boxes

In [10]:
boxes= split_boxes(board)
cv2.imshow("Box", boxes[0])
cv2.waitKey(0)
cv2.destroyAllWindows()  

In [11]:
test=np.array(boxes[0])
cropped_image = boxes[0][15:85, 15:85]

imgray_CI = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(imgray_CI, 150, 255, 0)

cv2.imshow("Box", imgray_CI)
cv2.waitKey(0)
cv2.destroyAllWindows() 

error: OpenCV(4.2.0) c:\projects\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xe227985e::Set<1,-1,-1>,struct cv::impl::A0xe227985e::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)


In [28]:
model=load_model('model-OCR.h5')

OSError: SavedModel file does not exist at: model-OCR.h5/{saved_model.pbtxt|saved_model.pb}

In [28]:
cropped_image.shape

(70, 70, 3)

In [38]:
boxes[0][15:85, 15:85]

array([[[0.63137255, 0.64313725, 0.7254902 ],
        [0.6627451 , 0.6745098 , 0.75686275],
        [0.68627451, 0.69803922, 0.78039216],
        ...,
        [0.64313725, 0.65490196, 0.7372549 ],
        [0.64705882, 0.65882353, 0.74117647],
        [0.65098039, 0.6627451 , 0.74509804]],

       [[0.61176471, 0.62352941, 0.70588235],
        [0.63529412, 0.64705882, 0.72941176],
        [0.6627451 , 0.6745098 , 0.75686275],
        ...,
        [0.65098039, 0.6627451 , 0.74509804],
        [0.65490196, 0.66666667, 0.74901961],
        [0.65490196, 0.66666667, 0.74901961]],

       [[0.58823529, 0.6       , 0.68235294],
        [0.61568627, 0.62745098, 0.70980392],
        [0.63921569, 0.65098039, 0.73333333],
        ...,
        [0.65882353, 0.67058824, 0.75294118],
        [0.65882353, 0.67058824, 0.75294118],
        [0.6627451 , 0.6745098 , 0.75686275]],

       ...,

       [[0.60784314, 0.61960784, 0.70196078],
        [0.59607843, 0.60784314, 0.69019608],
        [0.58039216, 0

In [31]:
cv2.imshow("Box", cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
cropped_image = boxes[0][15:85, 15:85]*255

In [62]:
intint=np.uint8(cropped_image)

In [56]:
cv2.imshow("Box", intint)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [63]:
imgray_CI = cv2.cvtColor(intint, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray_CI, 120, 255, 0)

cv2.imshow("Box", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [70]:
threshComp=cv2.resize(thresh,(28,28), interpolation = cv2.INTER_AREA)

In [71]:
cv2.imshow("Box", threshComp)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [60]:
thresh.shape

(28, 28)

In [152]:
def post_treatment_boxes(boxes):
    cleanBoxes=[]
    for box in boxes:
        cropped_image = np.uint8(box[15:85, 15:85]*255)
        imgray_CI = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        #thresh=imgray_CI
        ret, thresh = cv2.threshold(imgray_CI, 140, 255, 0)
        threshComp=cv2.resize(thresh,(28,28), interpolation = cv2.INTER_AREA)
        threshComp=abs(threshComp/255-1)
        cleanBoxes.append(threshComp)
    cleanBoxes=np.asarray(cleanBoxes)
    
    return cleanBoxes

In [153]:
cleanBoxes=post_treatment_boxes(boxes)

In [198]:
cv2.imshow("Box", cleanBoxes[10])
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [126]:
cleanBoxes=np.asarray(cleanBoxes)

In [97]:
cleanBoxes.shape

(81, 28, 28)

In [137]:
filePathModel="C:\\Users\\micka\\Desktop"
model=torch.load(filePathModel+"\\model2.pt")

In [138]:
class NetConv2(nn.Module):
    def __init__(self):
        super(NetConv2, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [199]:
testtest=cleanBoxes[10]
#testtest.reshape(-1,28,28)

In [200]:

testtest=torch.from_numpy(testtest)

testtest=Variable(testtest)
if testtest.data.std()!=0:
    testtest = testtest.sub_(testtest.data.mean()).div_(testtest.data.std()).float()
else:
    testtest=testtest.float()
testtest=torch.reshape(testtest,(-1,28,28))

In [201]:
output = model(testtest)
_,temp=torch.max(output.data,1)
temp

c:\python37\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([5])

In [196]:
output

tensor([[-10.0454,  -7.5149,  -5.2274,  -1.3623,  -2.4306,  -3.1729, -10.5402,
          -5.8389,  -6.7418,  -0.5042, -43.2281]],
       grad_fn=<LogSoftmaxBackward0>)

In [151]:
cleanBoxes[2]

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [na

In [163]:
testtest.data.std() == 0

tensor(True)

In [160]:
testtest=Variable(testtest)

TypeError: Variable data has to be a tensor, but got numpy.ndarray

In [161]:
testtest=torch.from_numpy(testtest)

testtest=Variable(testtest)